In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Importamos un csv del estudio que queremos realizar

In [2]:
df = pd.read_csv("./listing.csv")
df.head()

,game_date,country,tournament,season,home_field,home_team,away_team,home_team_score,away_team_score,home_team_score_extra_time,away_team_score_extra_time
0,2009-08-07,England,Championship,2009/2010,True,Middlesbrough,Sheffield Utd,0,0,NaN,NaN
1,2009-08-08,England,Championship,2009/2010,True,Preston,Bristol City,2,2,NaN,NaN
2,2009-08-08,England,League One,2009/2010,True,Yeovil,Tranmere,2,0,NaN,NaN
3,2009-08-08,England,League One,2009/2010,True,Southampton,Millwall,1,1,NaN,NaN
4,2009-08-08,England,League One,2009/2010,True,Gillingham,Swindon,5,0,NaN,NaN


#### Nos quedamos con las columnas que consideramos útiles para determinar la hipotesis:

Eliminamos las siguientes columnas:

    1.- country ya que todos son en inglaterra

    2.- home_field ya uqe no aporta informacion relevante

In [3]:
df1 = df[["game_date","tournament","season","home_team","away_team","home_team_score","away_team_score","home_team_score_extra_time","away_team_score_extra_time"]]

In [4]:
df1.head()

,game_date,tournament,season,home_team,away_team,home_team_score,away_team_score,home_team_score_extra_time,away_team_score_extra_time
0,2009-08-07,Championship,2009/2010,Middlesbrough,Sheffield Utd,0,0,NaN,NaN
1,2009-08-08,Championship,2009/2010,Preston,Bristol City,2,2,NaN,NaN
2,2009-08-08,League One,2009/2010,Yeovil,Tranmere,2,0,NaN,NaN
3,2009-08-08,League One,2009/2010,Southampton,Millwall,1,1,NaN,NaN
4,2009-08-08,League One,2009/2010,Gillingham,Swindon,5,0,NaN,NaN


In [5]:
premier_league = df1[df1.tournament == "Premier League"]
premier_league.head()

,game_date,tournament,season,home_team,away_team,home_team_score,away_team_score,home_team_score_extra_time,away_team_score_extra_time
106,2009-08-15,Premier League,2009/2010,Chelsea,Hull,2,1,NaN,NaN
107,2009-08-15,Premier League,2009/2010,Wolves,West Ham,0,2,NaN,NaN
108,2009-08-15,Premier League,2009/2010,Stoke,Burnley,2,0,NaN,NaN
109,2009-08-15,Premier League,2009/2010,Portsmouth,Fulham,0,1,NaN,NaN
110,2009-08-15,Premier League,2009/2010,Bolton,Sunderland,0,1,NaN,NaN


In [6]:
df2 = premier_league.rename(columns={
    'game_date':'Día',
    'tournament':'Campeonato',
    'season':'Temporada',
    'home_team':'Local',
    'away_team':'Visitante',
    'home_team_score':'GolesLocal',
    'away_team_score':'GolesVisitante',
    'home_team_score_extra_time':'GolesLocalProrroga',
    'away_team_score_extra_time':'GolesVisitanteProrroga',
})

In [7]:
df2.head()

,Día,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante,GolesLocalProrroga,GolesVisitanteProrroga
106,2009-08-15,Premier League,2009/2010,Chelsea,Hull,2,1,NaN,NaN
107,2009-08-15,Premier League,2009/2010,Wolves,West Ham,0,2,NaN,NaN
108,2009-08-15,Premier League,2009/2010,Stoke,Burnley,2,0,NaN,NaN
109,2009-08-15,Premier League,2009/2010,Portsmouth,Fulham,0,1,NaN,NaN
110,2009-08-15,Premier League,2009/2010,Bolton,Sunderland,0,1,NaN,NaN


In [8]:
pl = df2[["Día","Campeonato","Temporada","Local", "Visitante","GolesLocal","GolesVisitante"]]
pl.head()

,Día,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante
106,2009-08-15,Premier League,2009/2010,Chelsea,Hull,2,1
107,2009-08-15,Premier League,2009/2010,Wolves,West Ham,0,2
108,2009-08-15,Premier League,2009/2010,Stoke,Burnley,2,0
109,2009-08-15,Premier League,2009/2010,Portsmouth,Fulham,0,1
110,2009-08-15,Premier League,2009/2010,Bolton,Sunderland,0,1


In [9]:
pl_londres = pl[pl.Local == "Chelsea"]
pl_chealsea = pl_londres.reset_index(drop=True)
pl_chealsea

,Día,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante
0,2009-08-15,Premier League,2009/2010,Chelsea,Hull,2,1
1,2009-08-29,Premier League,2009/2010,Chelsea,Burnley,3,0
2,2009-09-20,Premier League,2009/2010,Chelsea,Tottenham,3,0
3,2009-10-04,Premier League,2009/2010,Chelsea,Liverpool,2,0
4,2009-10-24,Premier League,2009/2010,Chelsea,Blackburn,5,0
...,...,...,...,...,...,...,...
191,2019-08-31,Premier League,2019/2020,Chelsea,Sheffield Utd,2,2
192,2019-09-22,Premier League,2019/2020,Chelsea,Liverpool,1,2
193,2019-09-28,Premier League,2019/2020,Chelsea,Brighton,2,0
194,2019-10-19,Premier League,2019/2020,Chelsea,Newcastle,1,0


In [10]:
mes = list(pl_chealsea["Día"])
mes1 = ' '.join(mes)
mes2 = re.findall(r'\-\d{2}\-', mes1)
mes3 = [mes.strip("-") for mes in mes2]
mes4 = pd.DataFrame(mes3, columns = ["Mes"])
mes4.replace({
    '00':'UNKNOWN',
    '01':'Enero',
    '02':'Febrero',
    '03':'Marzo',
    '04':'Abril',
    '05':'Mayo',
    '06':'Junio',
    '07':'Julio',
    '08':'Agosto',
    '09':'Septiembre',
    '10':'Octubre',
    '11':'Noviembre',
    '12':'Dieciembre'
}, inplace = True)
mes_date = mes4
pl_chealsea["Día"] = mes_date
df_final = pl_chealsea.rename(columns={"Día":"Mes"})
df_final

,Mes,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante
0,August,Premier League,2009/2010,Chelsea,Hull,2,1
1,August,Premier League,2009/2010,Chelsea,Burnley,3,0
2,September,Premier League,2009/2010,Chelsea,Tottenham,3,0
3,October,Premier League,2009/2010,Chelsea,Liverpool,2,0
4,October,Premier League,2009/2010,Chelsea,Blackburn,5,0
...,...,...,...,...,...,...,...
191,August,Premier League,2019/2020,Chelsea,Sheffield Utd,2,2
192,September,Premier League,2019/2020,Chelsea,Liverpool,1,2
193,September,Premier League,2019/2020,Chelsea,Brighton,2,0
194,October,Premier League,2019/2020,Chelsea,Newcastle,1,0


In [11]:
df_final.head()

,Mes,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante
0,August,Premier League,2009/2010,Chelsea,Hull,2,1
1,August,Premier League,2009/2010,Chelsea,Burnley,3,0
2,September,Premier League,2009/2010,Chelsea,Tottenham,3,0
3,October,Premier League,2009/2010,Chelsea,Liverpool,2,0
4,October,Premier League,2009/2010,Chelsea,Blackburn,5,0


In [12]:
type(df_final["GolesLocal"])

pandas.core.series.Series

In [13]:
resultado = []
for i in df_final.iterrows():
    if i[1]['GolesLocal'] > i[1]['GolesVisitante']:
        resultado.append('1')
    elif i[1]['GolesLocal'] == i[1]['GolesVisitante']:
        resultado.append('x')
    else:
        resultado.append('2')
print(resultado)

['1', '1', '1', '1', '1', '1', '1', 'x', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', 'x', '1', 'x', '1', '2', '1', '1', '1', '1', '1', '1', 'x', '1', '1', '1', '1', '2', '2', '1', '1', 'x', '2', '1', 'x', '1', '1', 'x', '1', '1', '2', '1', '1', '1', '1', '1', '2', 'x', 'x', 'x', '1', '2', 'x', '1', '1', '1', '1', '1', '1', 'x', '1', '1', '1', '1', '1', '1', 'x', '1', '1', '1', '1', '1', 'x', '1', '1', '1', '1', '1', '2', 'x', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', 'x', '1', 'x', 'x', '1', '1', '1', 'x', '1', 'x', '2', '1', '2', '1', '2', '1', '2', '1', 'x', 'x', 'x', 'x', '1', 'x', 'x', '2', 'x', 'x', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', 'x', '2', '1', '1', '1', '1', '1', '1', '1', 'x', '2', '1', '1', '2', 'x', '1', 'x', '1', '1', '1', 'x', 'x', '1', 'x', '1', '1', '2', 'x', '1', '1', '1', 'x', '1', '1', 'x', '1', 'x', 'x', '2', '1', '1', '1']


In [14]:
df_final["Resultado"] = resultado

In [15]:
df_final

,Mes,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante,Resultado
0,August,Premier League,2009/2010,Chelsea,Hull,2,1,1
1,August,Premier League,2009/2010,Chelsea,Burnley,3,0,1
2,September,Premier League,2009/2010,Chelsea,Tottenham,3,0,1
3,October,Premier League,2009/2010,Chelsea,Liverpool,2,0,1
4,October,Premier League,2009/2010,Chelsea,Blackburn,5,0,1
...,...,...,...,...,...,...,...,...
191,August,Premier League,2019/2020,Chelsea,Sheffield Utd,2,2,x
192,September,Premier League,2019/2020,Chelsea,Liverpool,1,2,2
193,September,Premier League,2019/2020,Chelsea,Brighton,2,0,1
194,October,Premier League,2019/2020,Chelsea,Newcastle,1,0,1


In [16]:
pl

,Día,Campeonato,Temporada,Local,Visitante,GolesLocal,GolesVisitante
106,2009-08-15,Premier League,2009/2010,Chelsea,Hull,2,1
107,2009-08-15,Premier League,2009/2010,Wolves,West Ham,0,2
108,2009-08-15,Premier League,2009/2010,Stoke,Burnley,2,0
109,2009-08-15,Premier League,2009/2010,Portsmouth,Fulham,0,1
110,2009-08-15,Premier League,2009/2010,Bolton,Sunderland,0,1
...,...,...,...,...,...,...,...
27460,2019-11-09,Premier League,2019/2020,Tottenham,Sheffield Utd,1,1
27461,2019-11-09,Premier League,2019/2020,Southampton,Everton,1,2
27462,2019-11-09,Premier League,2019/2020,Newcastle,Bournemouth,2,1
27463,2019-11-09,Premier League,2019/2020,Burnley,West Ham,3,0


In [17]:
def createYear(date):
        year = re.search('\d+.',date).end()
        return date[:year - 1]
  
    
pl["Año"] = pl["Día"].apply(createYear)
pl["Año"]

<ipython-input-17-2f5238f2126e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl["Año"] = pl["Día"].apply(createYear)


106      2009
107      2009
108      2009
109      2009
110      2009
         ... 
27460    2019
27461    2019
27462    2019
27463    2019
27464    2019
Name: Año, Length: 3917, dtype: object

In [18]:
def createMonth(date):
        match = re.search('(\d+)\-([\d+]+)-([\d+]+)',date)
        if match:
            return match.group(2)
  
    
pl["Mes"] = pl["Día"].apply(createMonth)
pl["Mes"]

106      08
107      08
108      08
109      08
110      08
         ..
27460    11
27461    11
27462    11
27463    11
27464    11
Name: Mes, Length: 3917, dtype: object

In [19]:
def createDay(date):
        match = re.search('(\d+)\-([\d+]+)([\d+]+)',date)
        if match:
            return match.group(3)
  
    
pl["Día"] = pl["Día"].apply(createDay)
pl["Día"]

106      8
107      8
108      8
109      8
110      8
        ..
27460    1
27461    1
27462    1
27463    1
27464    1
Name: Día, Length: 3917, dtype: object

In [21]:
pl.columns

Index(['Día', 'Campeonato', 'Temporada', 'Local', 'Visitante', 'GolesLocal',
       'GolesVisitante', 'Año', 'Mes'],
      dtype='object')

In [22]:
pl_estudio = pl[["Año","Mes","Día","Campeonato","Local", "Visitante","GolesLocal","GolesVisitante"]]

In [23]:
pl_estudio

,Año,Mes,Día,Campeonato,Local,Visitante,GolesLocal,GolesVisitante
106,2009,08,8,Premier League,Chelsea,Hull,2,1
107,2009,08,8,Premier League,Wolves,West Ham,0,2
108,2009,08,8,Premier League,Stoke,Burnley,2,0
109,2009,08,8,Premier League,Portsmouth,Fulham,0,1
110,2009,08,8,Premier League,Bolton,Sunderland,0,1
...,...,...,...,...,...,...,...,...
27460,2019,11,1,Premier League,Tottenham,Sheffield Utd,1,1
27461,2019,11,1,Premier League,Southampton,Everton,1,2
27462,2019,11,1,Premier League,Newcastle,Bournemouth,2,1
27463,2019,11,1,Premier League,Burnley,West Ham,3,0


In [ ]:
mes4.replace({
    '00':'UNKNOWN',
    '01':'Enero',
    '02':'Febrero',
    '03':'Marzo',
    '04':'Abril',
    '05':'Mayo',
    '06':'Junio',
    '07':'Julio',
    '08':'Agosto',
    '09':'Septiembre',
    '10':'Octubre',
    '11':'Noviembre',
    '12':'Dieciembre'
}, inplace = True)